# Put Your Cassandra Python Driver On Steroids With Apache Arrow

## Topic of the day

```
+----------------------+  IO   +---------------+
| Batch of raw records | ----> | Analytics Job |
+----------------------+       +---------------+
```

![numpy](https://raw.githubusercontent.com/numpy/numpy/main/branding/logo/primary/numpylogo.png)

![arrow](https://arrow.apache.org/img/arrow.png)

![cassandra](https://miro.medium.com/max/1200/1*wzm0JlomFfsBF3mkZJQN7A.png)

In [1]:
%cd ..

/Users/aandres/source/cassarrow


In [2]:
import os

import cassandra.cluster
import cassandra.protocol
import humanize
import pandas as pd
import pyarrow as pa
import pyarrow.compute as pc
from pympler.asizeof import asizeof

import cassarrow
import cassarrow.impl

## Python Cassandra Driver

In [3]:
MIGRATION = """
CREATE TABLE IF NOT EXISTS cassarrow.time_series
(
    event_date      date,
    instrument_id   int,
    event_timestamp timestamp,
    value           double,
    PRIMARY KEY (event_date, instrument_id, event_timestamp)
);
"""

In [ ]:
QUERY = "SELECT * FROM time_series WHERE event_date = '2019-10-02'"

```
| event_date   |   instrument_id | event_timestamp     |      value |
|:-------------|----------------:|:--------------------|-----------:|
| 2019-10-02   |               1 | 2019-10-02 08:00:00 | 0.127755   |
| 2019-10-02   |               1 | 2019-10-02 08:15:00 | 0.256553   |
| 2019-10-02   |               1 | 2019-10-02 08:30:00 | 0.820371   |
| 2019-10-02   |               1 | 2019-10-02 08:45:00 | 0.711016   |
| 2019-10-02   |               1 | 2019-10-02 09:00:00 | 0.00108124 |
```

In [4]:
cluster = cassandra.cluster.Cluster()
session = cluster.connect("cassarrow")

In [6]:
results = list(session.execute(QUERY))

In [7]:
len(results)

36963

In [8]:
results[0]

Row(event_date=Date(18171), instrument_id=1, event_timestamp=datetime.datetime(2019, 10, 2, 8, 0), value=0.12775540988603173)

In [10]:
isinstance(results[0], tuple)

True

# 👎

In [11]:
sum(row.value for row in results) / len(results)

0.500400730152348

# 😵

In [12]:
%timeit sum(row.value for row in results) / len(results)

1.84 ms ± 12.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


# 🐢

In [13]:
actual_size = asizeof(results)
humanize.naturalsize(actual_size)

'13.3 MB'

```
| column          | type      |   bits |   bytes |
|:----------------|:----------|-------:|--------:|
| event_date      | date      |     32 |       4 |
| instrument_id   | int       |     32 |       4 |
| event_timestamp | timestamp |     64 |       8 |
| value           | double    |     64 |       8 |
| total           | -         |    192 |      24 |
```

In [51]:
expected_size = len(results) * 24
humanize.naturalsize(expected_size, True)

'866.3 KiB'

In [15]:
actual_size / expected_size

15.01936170404639

# 💥

In [16]:
type(results[0].event_date)

cassandra.util.Date

In [17]:
try:
    results[0].event_date.strftime("%Y-%m-%d")
except AttributeError as e:
    print(repr(e))

AttributeError("'Date' object has no attribute 'strftime'")


# 🤬

### In Summary:

* ❌ Not a great API for analytics
* ❌ Slow
* ❌ Uses a lot of memory
* ❌ Unusual types

## The solution: `cassarrow`, convert raw data directly to Apache Arrow

```
+-------------------+  Network   +----------+  Python Driver   +--------+
| Cassandra Cluster | ---------> | Raw Data | ---------------> | Tuples |
+-------------------+            +----------+                  +--------+


+-------------------+  Network   +----------+  C++ Code   +-------------+
| Cassandra Cluster | ---------> | Raw Data | ----------> | Arrow Table |
+-------------------+            +----------+             +-------------+


```

![Row to Column](https://arrow.apache.org/img/simd.png)

### Step 1: Convert the cassandra metadata to an Arrow `Schema`



#### Native types
```
| Cassandra   | pyarrow            | Note         |
|:------------|:-------------------|:-------------|
| ascii       | pa.string()        |              |
| bigint      | pa.int64()         |              |
| blob        | pa.binary()        |              |
| boolean     | pa.bool_()         |              |
| date        | pa.date32()        |              |
| decimal     |                    | Incompatible |
| double      | pa.float64()       |              |
| duration    | pa.duration("ns")  |              |
| float       | pa.float32()       |              |
| int         | pa.int32()         |              |
| smallint    | pa.int16()         |              |
| text        | pa.string()        |              |
| time        | pa.time64("ns")    |              |
| timestamp   | pa.timestamp("ms") |              |
| timeuuid    | pa.binary(16)      |              |
| tinyint     | pa.int8()          |              |
| uuid        | pa.binary(16)      |              |
| varchar     | pa.string()        |              |
| varint      |                    | Incompatible |
```

#### Collections / UDT
```
| Cassandra   | pyarrow   | Note   |
|:------------|:----------|:-------|
| list        | pa.list_  |        |
| map         | pa.map_   |        |
| set         | pa.list_  |        |
| udt         | pa.struct |        |
```

### Step 2: Converting the data

```
+------------------+     +----------------+           +--------------+     +-------+
|  Date32Builder   | --> |  Date32Array   | ------+-> | Record Batch | --> | Table |
+------------------+     +----------------+       |   +--------------+     +-------+
                                                  |
                                                  |
                                                  |
+------------------+     +----------------+       |
| TimestampBuilder | --> | TimestampArray | ------+
+------------------+     +----------------+       |
                                                  |
                                                  |
                                                  |
+------------------+     +----------------+       |
|  DoubleBuilder   | --> |  DoubleArray   | ------+
+------------------+     +----------------+       |
                                                  |
                                                  |
                                                  |
+------------------+     +----------------+       |
|   Int32Builder   | --> |   Int32Array   | ------+
+------------------+     +----------------+
```

### Demo

In [56]:
with cassarrow.install_cassarrow(session) as cassarrow_session:
    table = cassarrow.result_set_to_table(cassarrow_session.execute(QUERY))

In [48]:
type(table)

pyarrow.lib.Table

In [54]:
pc.mean(table["value"])

<pyarrow.DoubleScalar: 0.5004007301523449>

In [21]:
%timeit pc.mean(table['value']).as_py()

16.1 µs ± 223 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [22]:
humanize.naturalsize(table.nbytes)

'887.1 kB'

In [57]:
table.nbytes - expected_size

0

### Arrow summary

* ✅ Great API for analytics
* ✅ Fast
* ✅ Memory efficient
* ✅ No special/proprietary types

## Benchmark

### Executing a query

In [25]:
def execute_default(session, query):
    results = session.execute(query)
    return list(results)

In [26]:
def execute_cassarrow(session, query):
    with cassarrow.install_cassarrow(session) as cassarrow_session:
        results = cassarrow_session.execute(query)
    return cassarrow.result_set_to_table(results)

In [27]:
%timeit execute_default(session, QUERY)

167 ms ± 11.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
%timeit execute_cassarrow(session, QUERY)

108 ms ± 7.44 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [29]:
def get_binary(name: str) -> bytes:
    full_name = os.path.join("tests", "select", name)
    with open(full_name, "rb") as fp:
        return fp.read()


data = get_binary("time_series/0011.bin")

### Processing Raw Data

In [59]:
def parse_default(data: bytes) -> list[tuple]:
    msg_arrow = cassandra.protocol._ProtocolHandler.decode_message(
        5, {}, 3, 0, 8, data, None, []
    )
    return msg_arrow.parsed_rows

In [60]:
def parse_cassarrow(data) -> pa.RecordBatch:
    msg_arrow = cassarrow.impl.ArrowProtocolHandler.decode_message(
        5, {}, 3, 0, 8, data, None, []
    )
    return msg_arrow.parsed_rows

In [61]:
%timeit parse_default(data)

20.3 ms ± 116 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [62]:
%timeit parse_cassarrow(data)

2.09 ms ± 8.23 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# 👎 ➡️ 👍
# 😵 ➡️ 😌
# 🐢 ➡️ 🐇
# 💥 ➡️ 💧
# 🤬 ➡️ 🌞

# Conclusion

* Check the code on https://github.com/0x26res/cassarrow
* Install it: `pip install cassarrow`
* Apply the same approach to a similar problem!